# 复杂HTML解析

这一章我们将介绍解析复杂的HTML 页面的方法，从中抽取出我们需要
的信息。

### 2.1 不是一直都要用锤子

面对页面解析难题（Gordian Knot）的时候，不假思索地直接写几行语句来抽取信息是非
常直接的做法。但是，像这样鲁莽放纵地使用技术，只会让程序变得难以调试或脆弱不
堪，甚至二者兼具。**在开始解析网页之前，让我们看一些在解析复杂的HTML 页面时需要
避免的问题。**

- 寻找“打印此页”的链接，或者看看网站有没有HTML 样式更友好的移动版（把自己的请求头设置成处于移动设备的状态，然后接收网站移动版，更多内容在第12 章介绍）。


- 寻找隐藏在JavaScript 文件里的信息。要实现这一点，你可能需要查看网页加载的JavaScript 文件。我曾经要把一个网站上的街道地址（以经度和纬度呈现的）整理成格式整洁的数组时，查看过内嵌谷歌地图的JavaScript 文件，里面有每个地址的标记点。


- 虽然网页标题经常会用到，但是这个信息也许可以从网页的URL 链接里获取。


- 如果你要找的信息只存在于一个网站上，别处没有，那你确实是运气不佳。如果不只限于这个网站，那么你可以找找其他数据源。有没有其他网站也显示了同样的数据？网站上显示的数据是不是从其他网站上抓取后攒出来的？


**尤其是在面对埋藏很深或格式不友好的数据时，千万不要不经思考就写代码，一定要三思而后行。**如果你确定自己不能另辟蹊径，那么本章后面的内容就是为你准备的。

### 2.2 再端一碗BeautifulSoup

在这一节，我们将介绍通过**属性查找标签**的方法，**标签组**的使用，以及**标签解析树的导航过程**。

基本上，你见过的每个网站都会有层叠样式表（Cascading Style Sheet，CSS）.CSS 可以让HTML 元素呈现出差异化，使那些具有完全相同修饰的元素呈现出不同的样式。比如，有一些标签看起来是这样：
```CSS
<span class="green"></span>
```

而另一些标签看起来像这样：
```CSS
<span class="red"></span>
```

**网络爬虫可以通过class 属性的值，轻松地区分出两种不同的标签**。

例如，它们可以用BeautifulSoup 抓取网页上所有的红色文字，而绿色文字一个都不抓。*因为CSS 通过属性准确地呈现网站的样式，所以你大可放心，大多数新式网站上的class 和id 属性资源都非常丰富*。

In [9]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen('http://www.pythonscraping.com/pages/warandpeace.html')
bs = BeautifulSoup(html, 'html.parser')
print(bs)

<html>
<head>
<style>
.green{
	color:#55ff55;
}
.red{
	color:#ff5555;
}
#text{
	width:50%;
}
</style>
</head>
<body>
<h1>War and Peace</h1>
<h2>Chapter 1</h2>
<div id="text">
"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.</span>"
<p></p>
It was in July, 1805, and the speaker was the well-known <span class="green">Anna
Pavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya
Fedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man
of high rank and importance, who was the firs

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen('http://www.pythonscraping.com/pages/warandpeace.html')
bs = BeautifulSoup(html, 'html.parser')
nameList = bs.find_all('span', {'class': 'green'})
for name in nameList:
    print(name.get_text())

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


我们可以用findAll 函数抽取只包含在```<span class="green"></span> ```标签里的文字，这样就会得到一个人物名称的Python 列表（findAll 是一个非常灵
活的函数，我们后面会经常用到它）：

代码执行以后就会按照《战争与和平》中的人物出场顺序显示所有的人名。这是怎么实现
的呢？**之前，我们调用``bsObj.tagName`` 只能获取页面中的第一个指定的标签。现在，我们
调用``bsObj.findAll(tagName, tagAttributes) ``可以获取页面中所有指定的标签，不再只是
第一个了**。

In [10]:
titles = bs.find_all(['h1', 'h2','h3','h4','h5','h6'])
print([title for title in titles])

[<h1>War and Peace</h1>, <h2>Chapter 1</h2>]


In [11]:
allText = bs.find_all('span', {'class':{'green', 'red'}})
print([text for text in allText])

[<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.</span>, <span class="green">Anna
Pavlovna Scherer</span>, <span class="green">Empress Marya
Fedorovna</span>, <span class="green">Prince Vasili Kuragin</span>, <span class="green">Anna Pavlovna</span>, <span class="green">St. Petersburg</span>, <span class="red">If you have nothing better to do, Count [or Prince], and if the
prospect of spending an evening with a poor invalid is not too
terrible, I shall be very charmed to see you tonight between 7 and 10-
Annette Scherer.</span>, <span class="red">Heavens! w

```
什么时候使用get_text() 与什么时候应该保留标签？

.get_text() 会把你正在处理的HTML 文档中所有的标签都清除，然后返回
一个只包含文字的字符串。假如你正在处理一个包含许多超链接、段落和标
签的大段源代码，那么.get_text() 会把这些超链接、段落和标签都清除掉，
只剩下一串不带标签的文字。
用BeautifulSoup 对象查找你想要的信息，比直接在HTML 文本里查找信
息要简单得多。通常在你准备打印、存储和操作数据时，应该最后才使
用.get_text()。一般情况下，你应该尽可能地保留HTML 文档的标签结构。
```

### 2.2.1BeautifulSoup的find()和findAll()

BeautifulSoup 里的find() 和findAll() 可能是你最常用的两个函数。借助它们，你可以通
过标签的不同属性轻松地过滤HTML 页面，查找需要的标签组或单个标签。


这两个函数非常相似，BeautifulSoup 文档里两者的定义就是这样：
```
findAll(tag, attributes, recursive, text, limit, keywords)
find(tag, attributes, recursive, text, keywords)
```

很可能你会发现，自己在95% 的时间里都只需要使用前两个参数：``tag`` 和``attributes``。但是，我们还是应该仔细地观察所有的参数。

**标签参数tag 前面已经介绍过——你可以传一个标签的名称或多个标签名称组成的Python列表做标签参数**。例如，下面的代码将返回一个包含HTML 文档中所有标题标签的列表：
```
.findAll({"h1","h2","h3","h4","h5","h6"})
```

**属性参数``attributes``是用一个``Python ``字典封装一个标签的若干属性和对应的属性值**。例如，下面这个函数会返回HTML 文档里红色与绿色两种颜色的span 标签：
```
.findAll("span", {"class":{"green", "red"}})
```

**递归参数recursive 是一个布尔变量**。你想抓取HTML 文档标签结构里多少层的信息？**如果recursive 设置为True，findAll 就会根据你的要求去查找标签参数的所有子标签，以及子标签的子标签**。如果recursive 设置为False，findAll 就只查找文档的一级标签。

findAll默认是支持递归查找的（recursive 默认值是True）；一般情况下这个参数不需要设置，除非你真正了解自己需要哪些信息，而且抓取速度非常重要，那时你可以设置递归参数。

**文本参数text 有点不同，它是用标签的文本内容去匹配，而不是用标签的属性。**假如我们想查找前面网页中包含“the prince”内容的标签数量，我们可以把之前的findAll 方法换成下面的代码：
```
nameList = bsObj.findAll(text="the prince")
print(len(nameList))
```

In [13]:
nameList = bs.findAll(text="the prince")
print(len(nameList))

7


**范围限制参数limit，显然只用于findAll 方法**。find 其实等价于findAll 的limit 等于1 时的情形。如果你只对网页中获取的前x 项结果感兴趣，就可以设置它。但是要注意，这个参数设置之后，获得的前几项结果是按照网页上的顺序排序的，未必是你想要的那前几项。

**还有一个关键词参数keyword**，可以让你选择那些具有指定属性的标签。
```
allText = bsObj.findAll(id="text")
print(allText[0].get_text())
```

#### 2.2.2　其他BeautifulSoup对象

看到这里，你已经见过BeautifulSoup 库里的两种对象了.

- BeautifulSoup 对象:前面代码示例中的bsObj


- 标签Tag 对象:BeautifulSoup 对象通过find 和findAll，或者直接调用子标签获取的一列对象或单个对象，就像bsObj.div.h1


但是，这个库还有另外两种对象，虽然不常用，却应该了解一下。

- NavigableString 对象:用来表示标签里的文字，不是标签（有些函数可以操作和生成NavigableString 对象，而不是标签对象）。

- Comment 对象:用来查找HTML 文档的注释标签，``<!-- 像这样 -->``

这四个对象是你用BeautifulSoup 库时会遇到的所有对象（写作本书的时候）。

### 导航树

findAll 函数通过标签的名称和属性来查找标签 。但是如果你需要通过标签在文档中的位置来查找标签，该怎么办？这就是导航树（Navigating Trees）的作用。在第1 章里，我们看过用单一方向进行BeautifulSoup 标签树的导航：
```
bsObj.tag.subTag.anotherSubTa
```

现在我们用虚拟的在线购物网站http://www.pythonscraping.com/pages/page3.html 作为要抓
取的示例网页，演示HTML 导航树的纵向和横向导航（如图2-1 所示）。


这个HTML 页面可以映射成一棵树（为了简洁，省略了一些标签），如下所示：
```HTML
• html
  — body
    — div.wrapper
      — h1
      — div.content
      — table#giftList
        — tr
          — th
          — th
          — th
          — th
      — tr.gift#gift1
        — td
        — td
          — span.excitingNote
        — td
        — td
          — img
      — ……其他表格行省略了……
    — div.footer
```

- (1). 处理子标签和其他后代标签



- (2). 处理兄弟标签



- (3). 父标签处理

#### 1. 处理子标签和其他后代标签

一般情况下，BeautifulSoup 函数总是处理当前标签的后代标签。例如，bsObj.body.h1 选择了body 标签后代里的第一个h1 标签，不会去找body 外面的标签。

类似地，bsObj.div.findAll("img") 会找出文档中第一个div 标签，然后获取这个div 后代里所有的img 标签列表。

如果你只想找出子标签，可以用.children 标签：

In [18]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')

for child in bs.find('table', {'id': 'giftList'}).children:
    print(child)



<tr><th>
Item Title
</th><th>
Description
</th><th>
Cost
</th><th>
Image
</th></tr>


<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>


#### 2. 处理兄弟标签

``BeautifulSoup`` 的``next_siblings()`` 函数可以让收集表格数据成为简单的事情，尤其是处理带标题行的表格：

In [21]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen("http://www.pythonscraping.com/pages/page3.html")
bsObj = BeautifulSoup(html, 'html.parser')

for sibling in bsObj.find("table", {"id": "giftList"}).tr.next_siblings:
    print(sibling)



<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>


<tr class="gift" id="gift4"><td>
Dead Parrot
</td><td>
This is an ex-parr

例如，如果
我们选择一组标签中位于中间位置的一个标签，然后用next_siblings() 函数，那么它就
只会返回在它后面的兄弟标签。因此，选择标签行然后调用next_siblings，可以选择表
格中除了标题行以外的所有行。

#### 让标签的选择更具体
如果我们选择``bsObj.table.tr`` 或直接就用``bsObj.tr`` 来获取表格中的第一行，上面的代码也可以获得正确的结果。但是，我们还是采用更长的形式写了一
行代码，这可以避免各种意外：

``bsObj.find("table",{"id":"giftList"}).tr``

即使页面上只有一个表格（或其他目标标签），只用标签也很容易丢失细节。
另外，页面布局总是不断变化的。一个标签这次是在表格中第一行的位置，
没准儿哪天就在第二行或第三行了。**如果想让你的爬虫更稳定，最好还是让
标签的选择更加具体。如果有属性，就利用标签的属性**。


和next_siblings 一样， 如果你很容易找到一组兄弟标签中的最后一个标签， 那么
previous_siblings 函数也会很有用。


当然，还有next_sibling 和previous_sibling 函数，与next_siblings 和previous_siblings
的作用类似，只是它们返回的是单个标签，而不是一组标签。

#### 3. 父标签处理
在抓取网页的时候，查找父标签的需求比查找子标签和兄弟标签要少很多。通常情况
下，如果以抓取网页内容为目的来观察HTML 页面，我们都是从最上层标签开始的，然
后思考如何定位我们想要的数据块所在的位置。但是，偶尔在特殊情况下你也会用到
BeautifulSoup 的父标签查找函数，parent 和parents。

In [23]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')
print(bs.find('img',
              {'src':'../img/gifts/img1.jpg'})
      .parent.previous_sibling.get_text())


$15.00



这是如何实现的呢？下面的图形是我们正在处理的HTML 页面的部分结构，用数字表示步
骤的话：
```HTML
<tr>
— <td>
— <td>
— <td>(3)
   — "$15.00" (4)
— <td>(2)
   — <img src="../img/gifts/img1.jpg"> (1)
```

- (1) 选择图片标签src="../img/gifts/img1.jpg"；
- (2) 选择图片标签的父标签（在示例中是<td> 标签）；
- (3) 选择<td> 标签的前一个兄弟标签previous_sibling（在示例中是包含美元价格的<td>标签）；
- (4) 选择标签中的文字，“$15.00”。

### 2.3　正则表达式

计算机科学里曾经有个笑话：“如果你有一个问题打算用正则表达式（regular expression）来解决，那么就是两个问题了。”


之所以叫正则表达式，是因为它们可以识别正则字符串（regular string）；也就是说，它们可以这么定义：“如果你给我的字符串符合规则，我就返回它”，或者是“如果字符串不符合规则，我就忽略它”。这在要求快速浏览大文档，以查找像电话号码和邮箱地址之类的字符串时是非常方便的。


注意这里我用了一个词组正则字符串。什么是正则字符串？其实就是任意可以用一系列线性规则构成的字符串，就像：
- (1) 字母“a”至少出现一次；
- (2) 后面跟着字母“b”重复5 次；
- (3) 后面再跟字母“c”重复任意偶数次；
- (4) 最后一位是字母“d”，也可以没有。

正则表达式就是表达这组规则的缩写。这组规则的正则表达式如下所示：
```
aa*bbbbb(cc)*(d | )
```

### 2.4　正则表达式和BeautifulSoup

在抓取网页的时候，BeautifulSoup 和正则表达式总是配合使用的。其实，大多数支
持字符串参数的函数（比如，``find(id="aTagIdHere")``）都可以用正则表达式实现。

In [25]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')
images = bs.find_all('img',{'src': re.compile('\.\.\/img\/gifts/img.*\.jpg')})
for image in images:
    print(image['src'])

../img/gifts/img1.jpg
../img/gifts/img2.jpg
../img/gifts/img3.jpg
../img/gifts/img4.jpg
../img/gifts/img6.jpg


In [26]:
bs.find_all(lambda tag: len(tag.attrs) == 2)

[<img src="../img/gifts/logo.jpg" style="float:left;"/>,
 <tr class="gift" id="gift1"><td>
 Vegetable Basket
 </td><td>
 This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
 <span class="excitingNote">Now with super-colorful bell peppers!</span>
 </td><td>
 $15.00
 </td><td>
 <img src="../img/gifts/img1.jpg"/>
 </td></tr>,
 <tr class="gift" id="gift2"><td>
 Russian Nesting Dolls
 </td><td>
 Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
 </td><td>
 $10,000.52
 </td><td>
 <img src="../img/gifts/img2.jpg"/>
 </td></tr>,
 <tr class="gift" id="gift3"><td>
 Fish Painting
 </td><td>
 If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
 </td><td>
 $10,005.00
 </td><td>
 <img src="../img/gifts/img3.jpg"/>
 </td>

### 2.5　获取属性

到目前为止，我们已经介绍过如何获取和过滤标签，以及获取标签里的内容。但是，在网
络数据采集时你经常不需要查找标签的内容，而是需要查找标签属性。比如标签``<a> ``指向
的URL 链接包含在href 属性中，或者```<img> ```标签的图片文件包含在src 属性中，这时获
取标签属性就变得非常有用了。

对于一个标签对象，可以用下面的代码获取它的全部属性：
```
myTag.attrs
```

要注意这行代码返回的是一个Python 字典对象，可以获取和操作这些属性。比如要获取图
片的资源位置src，可以用下面这行代码：
```
myImgTag.attrs["src"]
```

### 2.6　Lambda表达式

Lambda 表达式本质上就是一个函数，可以作为其他函数的变量使用；也就是说，一个函
数不是定义成``f(x, y)``，而是定义成``f(g(x), y)``，或``f(g(x), h(x)) ``的形式

BeautifulSoup 允许我们把特定函数类型当作findAll 函数的参数。唯一的限制条件是这些
函数必须把一个标签作为参数且返回结果是布尔类型。BeautifulSoup 用这个函数来评估它
遇到的每个标签对象，最后把评估结果为“真”的标签保留，把其他标签剔除。

```
soup.findAll(lambda tag: len(tag.attrs) == 2)
```
这行代码会找出下面的标签：
```
<div class="body" id="content"></div>
<span style="color:red" class="title"></span>
```

如果你愿意多写一点儿代码，那么在BeautifulSoup 里用Lambda 表达式选择标签，将是正
则表达式的完美替代方案。

In [28]:
bs.find_all(lambda tag: tag.get_text() == 'Or maybe he\'s only resting?')

[<span class="excitingNote">Or maybe he's only resting?</span>]

In [29]:
bs.find_all('', text='Or maybe he\'s only resting?')

["Or maybe he's only resting?"]